In [21]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [2]:
df = pd.read_csv("data/2023/Divvy_Trips_2023_full.csv.gz", compression="gzip")

In [3]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member


# Критерии нахождения станции с высокой маятниковостью
Будем считать что есть несколько критериев, которые позволяют определить станцию с высокой маятниковостью:
- **Большая разница между приехавшими/уехавшими велосипедами утром и вечером**: если разница между этими показателями большая то это может означать что велосипеды уезжают в другие районы города, а возвращаются только вечером. И наоборот - если разница между показателями маленькая, значит нет четкого распределения поездок в/с этой станции.
- **Большое количество коротких поездок**: обычно маятниковые поездки связаны с небольшими перемещениями (например, с работы домой и назад) по городу, а не с длинными прогулочными поездками. Так что можно сделать вывод - если в станции много коротких поездок, то это может быть признаком маятниковости.

Попробуем рассчитать эти показатели для каждой станции и найти станции с самой ярко выраженной маятниковостью.

In [4]:
# Считаем сколько поездок началось/закончилось в каждой станции
start_count = df.groupby("start_station_name")["ride_id"].count().reset_index()
start_count.columns = ["station_name", "start_count"]
end_count = df.groupby("end_station_name")["ride_id"].count().reset_index()
end_count.columns = ["station_name", "end_count"]
station_counts = pd.merge(start_count, end_count, on="station_name", how="outer")
station_counts = station_counts.fillna(0)
station_counts["total_count"] = (
    station_counts["start_count"] + station_counts["end_count"]
)


## Считаем сколько поездок началось/закончилось утром/вечером


In [5]:
# Получаем час начала и конца поездки
df["start_hour"] = pd.to_datetime(df["started_at"]).dt.hour
df["end_hour"] = pd.to_datetime(df["ended_at"]).dt.hour

# Разделяем утренние и вечерние поездки
morning_trips = df[(df["start_hour"] >= 6) & (df["start_hour"] < 12)]
evening_trips = df[(df["end_hour"] >= 15) & (df["end_hour"] < 20)]

# Получаем сколько приехало/уехало с каждой станции утром
morning_out_counts = (
    morning_trips.groupby("start_station_name")["ride_id"].count().reset_index()
)
morning_out_counts.columns = ["station_name", "morning_in_count"]
morning_in_counts = (
    morning_trips.groupby("end_station_name")["ride_id"].count().reset_index()
)
morning_in_counts.columns = ["station_name", "morning_out_count"]
morning_counts = pd.merge(
    morning_in_counts, morning_out_counts, on="station_name", how="outer"
)

# Получаем сколько приехало/уехало с каждой станции вечером
evening_in_counts = (
    evening_trips.groupby("end_station_name")["ride_id"].count().reset_index()
)
evening_in_counts.columns = ["station_name", "evening_in_count"]
evening_out_counts = (
    evening_trips.groupby("start_station_name")["ride_id"].count().reset_index()
)
evening_out_counts.columns = ["station_name", "evening_out_count"]
evening_counts = pd.merge(
    evening_in_counts, evening_out_counts, on="station_name", how="outer"
)

# Объединяем датасеты
station_counts = pd.merge(
    station_counts, morning_counts, on="station_name", how="outer"
)
station_counts = pd.merge(
    station_counts, evening_counts, on="station_name", how="outer"
)
station_counts = station_counts.fillna(0)


## Считаем длительность поездки и количество коротких поездок

In [6]:
# Продолжительность поездки
df["tripduration"] = (
    pd.to_datetime(df["ended_at"]) - pd.to_datetime(df["started_at"])
).dt.total_seconds()
short_trips = df[df["tripduration"] <= 30 * 60]
# Количество коротких поездок
short_counts = (
    short_trips.groupby("start_station_name")["ride_id"].count().reset_index()
)
short_counts.columns = ["station_name", "short_count"]
station_counts = pd.merge(station_counts, short_counts, on="station_name", how="outer")
station_counts = station_counts.fillna(0)

In [7]:
# Считаем долю коротких поездок
station_counts["short_ratio"] = (
    station_counts["short_count"] / station_counts["total_count"]
)

In [8]:
# Считаем общее количество поездок утром и вечером
station_counts["morning_count"] = (
    station_counts["morning_in_count"] + station_counts["morning_out_count"]
)
station_counts["evening_count"] = (
    station_counts["evening_in_count"] + station_counts["evening_out_count"]
)

# Считаем разницу между количеством поездок, начавшихся и закончившихся в станции утром и вечером
station_counts["morning_swing"] = abs(
    station_counts["morning_in_count"] - station_counts["morning_out_count"]
)
station_counts["evening_swing"] = abs(
    station_counts["evening_in_count"] - station_counts["evening_out_count"]
)
station_counts["swing"] = (
    station_counts["morning_swing"] + station_counts["evening_swing"]
)

## Находим станции с высокой маятниковостью
 - Много коротких поездок

Cравнение двух показателей утром и вечером:
- (Утром со станции больше уезжают чем приезжают) И (вечером на станцию больше приезжают чем уезжают)
- (Утром на станцию больше приезжают чем уезжают) И (вечером со станции больше уезжают чем приезжают)

In [15]:
swing_stations = station_counts[
    (station_counts["swing"] > 10)
    & (
        (
            (station_counts["morning_out_count"] > station_counts["morning_in_count"])
            & (station_counts["evening_out_count"] < station_counts["evening_in_count"])
        )
        | (
            (station_counts["morning_out_count"] < station_counts["morning_in_count"])
            & (station_counts["evening_out_count"] > station_counts["evening_in_count"])
        )
    )
    & (station_counts["short_ratio"] > 0.3)
]
swing_stations = swing_stations.sort_values(by=["swing"], ascending=False)

In [18]:
swing_stations.head(20)

,station_name,start_count,end_count,total_count,morning_out_count,morning_in_count,evening_in_count,evening_out_count,short_count,short_ratio,morning_count,evening_count,morning_swing,evening_swing,swing
182,Columbus Dr & Randolph St,2191.0,1781.0,3972.0,642.0,833.0,731.0,852.0,2044.0,0.514602,1475.0,1583.0,191.0,121.0,312.0
328,Halsted St & Clybourn Ave,2465.0,2586.0,5051.0,567.0,544.0,1334.0,1050.0,2404.0,0.475945,1111.0,2384.0,23.0,284.0,307.0
592,Pine Grove Ave & Waveland Ave,2046.0,1870.0,3916.0,492.0,694.0,728.0,780.0,1868.0,0.477017,1186.0,1508.0,202.0,52.0,254.0
149,Clark St & Chicago Ave,1507.0,1179.0,2686.0,325.0,518.0,537.0,565.0,1451.0,0.540208,843.0,1102.0,193.0,28.0,221.0
1054,Wentworth Ave & Cermak Rd,887.0,961.0,1848.0,304.0,153.0,412.0,386.0,792.0,0.428571,457.0,798.0,151.0,26.0,177.0
870,Racine Ave & Fullerton Ave,1871.0,1912.0,3783.0,575.0,408.0,723.0,716.0,1826.0,0.482686,983.0,1439.0,167.0,7.0,174.0
216,Damen Ave & Pierce Ave,2389.0,2499.0,4888.0,789.0,668.0,1030.0,981.0,2306.0,0.471768,1457.0,2011.0,121.0,49.0,170.0
520,Michigan Ave & 8th St,1213.0,1046.0,2259.0,358.0,489.0,385.0,423.0,1084.0,0.479858,847.0,808.0,131.0,38.0,169.0
911,Sedgwick St & Huron St,1560.0,1350.0,2910.0,521.0,563.0,478.0,600.0,1538.0,0.528522,1084.0,1078.0,42.0,122.0,164.0
878,Ravenswood Ave & Irving Park Rd,703.0,719.0,1422.0,206.0,163.0,359.0,262.0,683.0,0.480309,369.0,621.0,43.0,97.0,140.0


# Нанесем результат на карту

In [19]:
# Получим координаты станций
coords = pd.read_csv("data/2023/station_coord.csv")

In [20]:
# Объединим датасеты
swing_stations = pd.merge(swing_stations, coords, on="station_name", how="left")

In [26]:
swing_stations = swing_stations.dropna(subset=["lat", "lng"])

In [51]:
# Нанесем результат на карту
chicago = folium.Map(location=[41.85, -87.65], zoom_start=12)
marker_cluster = MarkerCluster().add_to(chicago)
for i in range(len(swing_stations)):
    folium.CircleMarker(
        location=(swing_stations.iloc[i]["lat"], swing_stations.iloc[i]["lng"]),
        fill_color="#990066",
        color="gray",
        fill_opacity=0.9,
        popup=f"Name: {swing_stations.iloc[i]['station_name']}<br>Swing: {swing_stations.iloc[i]['swing']}<br>Morning swing: {swing_stations.iloc[i]['morning_swing']}<br>Evening swing: {swing_stations.iloc[i]['evening_swing']}<br>Short ratio: {round(swing_stations.iloc[i]['short_ratio'], 2)}",
        parse_html=True,
    ).add_to(marker_cluster)

In [52]:
chicago